# Data preparation

#### Requirements

In [ ]:
!pip install pandas
!pip install google-cloud-bigquery
!pip install google-cloud-storage
!pip install pandas-gbq


En este apartado lo que queremos conseguir es tratar los datos para que se los podamos pasar al modelo, tanto para entrentarlo como para clasificar neuvos elementos posteriormente.


Vamos a hacer tres tratamientos de datos distintos que depende del tipo de clasificación que queremos obtener: mensaje de cookies, aceptar cookies y cerrar cookies.

Una vez sabemos que datos vamos a querer tener para nuestro modelaje, los crearemos y los guardaremos como objetos pickle para el posterior modelaje.

## Mensaje de cookies

Como vimos anteriormente los mensajes de cookies siempre son del tipo (name en nuestros datos) div, span, td o p.

In [ ]:
%%time
import pandas as pd
import os

credentials_file = 'tfmfabri-2cf8db9433ba.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_file

from google.cloud import bigquery
client = bigquery.Client()

# Query sin limitar (todos los datos)
query = 'SELECT label,text,url FROM tfm.tfm_url_df where name in ("div","p","span","td") and text is not null and is_in_head = false and is_script = false and is_no_script = false and width_height != "(0, 0)"' 

# Query sin limitada (solo 100000 filas)
#query = 'SELECT label,text,url FROM tfm.tfm_url_df where name in ("div","p","span","td") and text is not null and is_in_head = false and is_script = false and is_no_script = false and width_height != "(0, 0)" LIMIT 100000' 

df = pd.read_gbq(query,project_id='tfmfabri')


In [ ]:
# Quitamos caracteres especiales por medios de expresiones regulares
import re
df['text'] = df['text'].apply(lambda x: re.sub("[≡!@#$≡<>+%*()'-]|\n|\t", ' ', x))

In [ ]:
# Quitamos los espacios en blanco duplicados
df['text'] = df['text'].apply(lambda x: re.sub(' +', ' ',x))

In [ ]:
# Calculamos la longitud del texto
df['text_length'] = df['text'].apply(lambda x: len(x.split()))

In [ ]:
# Quitamos las filas que no tengan texto
df = df[df['text_length'] != 0]
df.count()

Nos hemos descargado 100000 filas de nuestra base de datos, sin embargo, para poder saber con certeza como son nuestros elementos "mensaje_cookies" vamos a descargarnos todos y los vamos a guardar en el dataframe df_m.

In [ ]:
query = 'SELECT label,text,url FROM tfm.tfm_url_df where name in ("div","p","span","td") and text is not null and is_in_head = false and is_script = false and is_no_script = false and width_height != "(0, 0)" and label = "mensaje_cookies"' 

df_m = pd.read_gbq(query,project_id='tfmfabri')


In [ ]:
# Calculamos la longitud del texto de los mensajes de cookies
df_m['text_length'] = df_m['text'].apply(lambda x: len(x.split()))

In [ ]:
# Vemos los distintos valores que toma el campo "text_length"
df_m['text_length'].describe()

In [ ]:
# Eliminamos las filas de nuestro dataframe de datos (df) donde la lontiud del texto sea un 25% 
# mayor que el máximo de la longitud del texto de los mensajes de cookies

max_cookie_message_length = df_m['text_length'].max() * 1.25
df = df[(df['text_length'] < max_cookie_message_length) & (df['text_length'] > 2)]
df.count()

In [ ]:
df['text'].nunique()

In [ ]:
# Borramos los duplicados
df = df.sort_values(by=['label'],ascending=False).drop_duplicates(keep='first')

### Creamos los datasets test y train para entrenar y testar el modelo
Debido a la gran discrepancia en el número de datos dependiendo de si es mensaje cookies o si no lo es, vamos a filtrar y vamos a pasarle al modelo un número similar de cada tipo.

In [ ]:
df = df[['label','text']]
df_message = df[df['label'] == 'mensaje_cookies']
df_message_train = df_message.sample(frac=0.8)
df_message_test = pd.concat([df_message, df_message_train]).drop_duplicates(keep=False)

df_other = df[df['label'] == '0'].sample(frac=0.01) #Dejo solo el 10%
df_other_train = df_other.sample(frac=0.8)
df_other_test = pd.concat([df_other, df_other_train]).drop_duplicates(keep=False)

df_train = pd.concat([df_message_train,df_other_train])
df_test = pd.concat([df_message_test,df_other_test])


In [ ]:
# Número de mensajes de cookies en train
len(df_message_train.index)

In [ ]:
# Número de datos no mensaje de cookies en test
len(df_message_test.index)

In [ ]:
# Número de datos totales en train
len(df_train.index)

In [ ]:
# Número de datos totales en test
len(df_test.index)

In [ ]:
# Guardamos en un pickle
import pickle

with open('df_message_train.pkl', 'wb') as f:
    pickle.dump(df_train, f)
    
with open('df_message_test.pkl', 'wb') as f:
    pickle.dump(df_test, f)

## Aceptar cookies

Como vimos anteriormente los mensajes de cookies siempre son del tipo (name en nuestros datos) div, span, td o p.

In [ ]:
%%time
import pandas as pd
import os

credentials_file = 'tfmfabri-2cf8db9433ba.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_file

from google.cloud import bigquery
client = bigquery.Client()

# Query sin limitar (todos los datos)
query = 'SELECT * FROM tfm.tfm_url_df where name in ("a","b","button","div","label","span","strong","td","p") and width_height not like "%None%" and is_in_head = false and is_script = false and is_no_script = false and width_height != "(0, 0)" and position_x_y != "(0, 0)"' 

# Query sin limitada (solo 100000 filas)
#query = 'SELECT * FROM tfm.tfm_url_df where name in ("a","b","button","div","label","span","strong","td","p") and width_height not like "%None%" and is_in_head = false and is_script = false and is_no_script = false and width_height != "(0, 0)" and position_x_y != "(0, 0)" LIMIT 100000' 

df = pd.read_gbq(query,project_id='tfmfabri')


df.loc[~df['label'].isin(['aceptar_cookies','mensaje_cookies']),'label'] = '0'

In [ ]:
df['label'].unique()

In [ ]:
len(df.index)

Los campos que vamos a introducir a nuestro modelo clasificador de aceptar cookies los vamos a calcular por URL, esto es debido a que hay campos que se calculan con respecto al elemento "mensaje de cookies" que es único y distinto para cada página.

#### Declaramos las funciones que vamos a utilizar


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import math

# Función que dada una fila coge su posición x y su ancho y calcula la nueva coordenada x que será más real
# pues estará en el centro del elemento
def new_x(x):
    return x['position_x'] + x['width']/2

# Función que dada una fila coge su posición y y su alto y calcula la nueva coordenada y que será más real
# pues estará en el centro del elemento
def new_y(y):
    return y['position_y'] + y['height']/2

# Calula la distancia entre dos puntos. Objetivo: calcular la distancia entre el elemento y el elemento
# "mensaje cookies"
# La funcion tiene como argumentos: una fila del dataframe de donde sacaremos las coordenadas y la coordenada
# x e y del elemento "mensaje cookies"

def calculate_distance(x,cm_x,cm_y):
    return math.sqrt((x['position_x'] - cm_x)**2 + (x['position_y'] - cm_y)**2)

# Listado de palabras con las que vamos a hacer un one hot encoding. Son las palabras que más están presentes
# en los botones de aceptar cookies.

word_list = {
'de',
'política',
'y',
'a',
'en',
'cookies',
'más',
'la',
'ver',
'al',
'información',
'que',
'aceptar',
'ok',
'acepto',
'cookies',
'de',
'continuar',
'estoy',
'acuerdo',
'entendido',
'navegando',
'seguir'
}

# Función a la que se le pasa un elemento, se coge el texto y se crea una matriz one hot encoding.
def is_in_one_hot(x):

    text = x['text'].lower().strip()
    text = re.sub(' +', ' ',text)
    text = text.split()
    if len(text) < 7:
        for t in text:
            if t in word_list:
                x[t] = 1
    return x

# Función que calcula la distancia al padre común mas cercano de dos elementos HTML
def get_parent_distance(s_cm, s_el):

   
    s_cm = s_cm[0].split(">")
    s_cm = [w.strip() for w in s_cm]
    s_cm_len = len(s_cm)
    
    s_el = s_el.split(">")
    s_el = [w.strip() for w in s_el]   
    s_el_len = len(s_el)
    
    count_parent = 0

    if s_cm_len > s_el_len:
        smaller = min(s_cm_len,s_el_len)
        s_cm = s_cm[:smaller]
        s_el = s_el[:smaller]
        count_parent = s_cm_len - s_el_len
    else:
        smaller = min(s_cm_len,s_el_len)
        s_cm = s_cm[:smaller]
        s_el = s_el[:smaller]
        
    count = smaller

    while s_cm != s_el:
        count_parent += 1
        smaller -= 1
        s_cm = s_cm[:smaller]
        s_el = s_el[:smaller]        

    return count_parent




#### Aplicamos las funciones

In [ ]:
df_f = pd.DataFrame()

unique_url = df['url'].unique()


for url in unique_url:
    df_u = df[df['url'] == url]
    if df_u.count()[0] == 0:
        continue
    if df_u[df_u['label'] == 'mensaje_cookies'].count()[0] == 0:
        continue
        
    df_u['text'] = df_u['text'].apply(lambda x: re.sub("[≡!@#$≡<>+%*()'-]|\n|\t", ' ', str(x)))
    df_u['text'] = df_u['text'].apply(lambda x: re.sub(' +', ' ',x))
    df_u['text_length'] = df_u['text'].apply(lambda x: len(x.split()))
    
    df_u['position_x_y'] = df_u['position_x_y'].apply(lambda x: x.replace("(",""))
    df_u['position_x_y'] = df_u['position_x_y'].apply(lambda x: x.replace(")",""))
    df_u['position_x'] = df_u['position_x_y'].apply(lambda x: x.split(",")[0])
    df_u['position_x'] = df_u['position_x'].astype(float)
    df_u['position_y'] = df_u['position_x_y'].apply(lambda x: x.split(",")[1])
    df_u['position_y'] = df_u['position_y'].astype(float)
    df_u['width_height'] = df_u['width_height'].apply(lambda x: x.replace("(",""))
    df_u['width_height'] = df_u['width_height'].apply(lambda x: x.replace(")",""))
    df_u['width'] = df_u['width_height'].apply(lambda x: x.split(",")[0])
    df_u['width'] = df_u['width'].astype(float)
    df_u['height'] = df_u['width_height'].apply(lambda x: x.split(",")[1])
    df_u['height'] = df_u['height'].astype(float)
    df_u['position_x'] = df_u.apply(lambda x: new_x(x) , axis=1)
    df_u['position_y'] = df_u.apply(lambda x: new_y(x) , axis=1)

    cm_y = df_u['position_y'][df_u['label'] == 'mensaje_cookies'].values[0]
    cm_x = df_u['position_x'][df_u['label'] == 'mensaje_cookies'].values[0]

    df_u['distance_cm'] = df_u.apply(lambda x: calculate_distance(x,cm_x,cm_y),axis=1)
    df_f = pd.concat([df_f,df_u])

# Los siguientes cálculos los hacemos con el dataframe entero, pero antes comprobamos que no está vacío.
if len(df_f.index) > 0:
    
    # Seleccionamos las columnas que queremos del dataframe principal (df_f) y las guardamos en df_d
    df_d = df_f[['css_selector','url','label','text_length','width','height','distance_cm','text','name']]
    
    # Aplicamos one hot enconding al texto
    df_d = df_d.apply(lambda x: is_in_one_hot(x),axis=1)
    
    # Calculamos la mínima distancia al elemento HTML padre común entre el elemento a probar y el elemento de mensaje de cookies.
    # Este cálculo lo volvemos a hacer por URl
    df_d['parent_distance'] = 0
    for url in df_d['url'].unique():
        df_url = df_d[df_d['url'] == url]
        cm = df_d[df_d['url'] == url][df_d['label'] == 'mensaje_cookies']
        cm_css_selector = cm['css_selector'].values
        df_url['parent_distance'] = df_url['css_selector'].apply(lambda x: get_parent_distance(cm_css_selector,x))
        df_d[df_d['url'] == url] = df_url
    
    df_d = df_d.fillna(0)

#### Normalización de datos

In [ ]:
# Normalize data
from sklearn import preprocessing

columns_to_normalize = ['text_length','width','height','distance_cm','parent_distance']

for col in columns_to_normalize:
    df_d[col]=(df_d[col]-df_d[col].min())/(df_d[col].max()-df_d[col].min())
    #df_d[col]=(df_d[col]-df_d[col].mean())/df_d[col].std()

In [ ]:
df_d.describe()

In [ ]:
df_d[df_d['label'].isin(['aceptar_cookies'])].describe()

In [ ]:
df_tt = df_d.copy()

df_tt[df_tt['name'] == 'a']['name'] = None

# Hacemos get dummies para crear las columnas one hot encoding de los atributos name de los elementos
df_tt = pd.get_dummies(df_tt, columns=['name'])

# Quitamos las columnas que no necesitamos
df_tt.drop('css_selector',axis=1, inplace=True)
df_tt.drop('url',axis=1, inplace=True)
df_tt.drop('text',axis=1, inplace=True)

# Seleccionamos un parte de los elementos que están categorizados como "aceptar_cookies"
df_tt_ok = df_tt[df_tt['label'].isin(['aceptar_cookies'])]
df_tt_ok_test = df_tt_ok.sample(frac=0.2)
df_tt_ok_train = pd.concat([df_tt_ok, df_tt_ok_test]).drop_duplicates(keep=False)

# Seleccionamos una parte de los elementos que no son "aceptar_cookies"
# Aquí dividimos en dos partes, para entrenar bien a nuestro modelo, le metemos elementos de cualquier sitio
# de la página y elemento que están cercanos al "mensaje_cookies" así le enseñamos a saber que 
# no por estar cerca es un elemento del tipo "aceptar_cookies"
df_tt_ko = df_tt[~df_tt['label'].isin(['aceptar_cookies'])]
df_tt_ko_near = df_tt_ko[df_tt_ko['parent_distance'] < 0.17].sample(frac=0.010)
df_tt_ko_far = df_tt_ko[df_tt_ko['parent_distance'] >= 0.17].sample(frac=0.003)



df_tt_ko = pd.concat([df_tt_ko_near, df_tt_ko_far]).sample(frac=0.6)



df_tt_ko_test = df_tt_ko.sample(frac=0.2)
df_tt_ko_train = pd.concat([df_tt_ko, df_tt_ko_test]).drop_duplicates(keep=False)


df_tt_test = pd.concat([df_tt_ok_test,df_tt_ko_test])
df_tt_test_label = df_tt_test.copy()['label']
df_tt_test.drop('label',axis=1, inplace=True)


df_tt_train = pd.concat([df_tt_ok_train,df_tt_ko_train])
df_tt_train_label = df_tt_train.copy()['label']
df_tt_train.drop('label',axis=1, inplace=True)

df_tt_total = pd.concat([df_tt_ok,df_tt_ko.sample(frac=0.001)])
df_tt_total_label = df_tt_total.copy()['label']
df_tt_total.drop('label',axis=1, inplace=True)


In [ ]:
# Guardamos en un pickle
import pickle

with open('df_aceptar_train.pkl', 'wb') as f:
    pickle.dump(df_tt_train, f)
    
with open('df_aceptar_test.pkl', 'wb') as f:
    pickle.dump(df_tt_test, f)
    
with open('df_aceptar_train_label.pkl', 'wb') as f:
    pickle.dump(df_tt_train_label, f)
    
with open('df_aceptar_test_label.pkl', 'wb') as f:
    pickle.dump(df_tt_test_label, f)

## Cerrar cookies

Como vimos anteriormente los mensajes de cookies siempre son del tipo (name en nuestros datos) div, span, td o p.

In [ ]:
%%time
import pandas as pd
import os

credentials_file = 'tfmfabri-2cf8db9433ba.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_file

from google.cloud import bigquery
client = bigquery.Client()

# Query sin limitar (todos los datos)
query = 'SELECT * FROM tfm.tfm_url_df where name in ("a","b","button","div","label","span","strong","td","p") and width_height not like "%None%" and is_in_head = false and is_script = false and is_no_script = false and width_height != "(0, 0)" and position_x_y != "(0, 0)"' 

# Query sin limitada (solo 100000 filas)
#query = 'SELECT * FROM tfm.tfm_url_df where name in ("a","b","button","div","label","span","strong","td","p") and width_height not like "%None%" and is_in_head = false and is_script = false and is_no_script = false and width_height != "(0, 0)" and position_x_y != "(0, 0)" LIMIT 100000' 

df = pd.read_gbq(query,project_id='tfmfabri')


df.loc[~df['label'].isin(['cerrar_cookies','mensaje_cookies']),'label'] = '0'

In [ ]:
df['label'].unique()

In [ ]:
len(df.index)

Los campos que vamos a introducir a nuestro modelo clasificador de cerrar cookies los vamos a calcular por URL, esto es debido a que hay campos que se calculan con respecto al elemento "mensaje de cookies" que es único y distinto para cada página.

#### Declaramos las funciones que vamos a utilizar


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import math

# Función que dada una fila coge su posición x y su ancho y calcula la nueva coordenada x que será más real
# pues estará en el centro del elemento
def new_x(x):
    return x['position_x'] + x['width']/2

# Función que dada una fila coge su posición y y su alto y calcula la nueva coordenada y que será más real
# pues estará en el centro del elemento
def new_y(y):
    return y['position_y'] + y['height']/2

# Calula la distancia entre dos puntos. Objetivo: calcular la distancia entre el elemento y el elemento
# "mensaje cookies"
# La funcion tiene como argumentos: una fila del dataframe de donde sacaremos las coordenadas y la coordenada
# x e y del elemento "mensaje cookies"

def calculate_distance(x,cm_x,cm_y):
    return math.sqrt((x['position_x'] - cm_x)**2 + (x['position_y'] - cm_y)**2)

# Listado de palabras con las que vamos a hacer un one hot encoding. Son las palabras que más están presentes
# en los botones.

word_list = {
'de',
'política',
'y',
'a',
'en',
'cookies',
'más',
'la',
'ver',
'al',
'información',
'que',
'aceptar',
'ok',
'acepto',
'cookies',
'de',
'continuar',
'estoy',
'acuerdo',
'entendido',
'navegando',
'seguir'
}

# Función a la que se le pasa un elemento, se coge el texto y se crea una matriz one hot encoding.
def is_in_one_hot(x):

    text = x['text'].lower().strip()
    text = re.sub(' +', ' ',text)
    text = text.split()
    if len(text) < 7:
        for t in text:
            if t in word_list:
                x[t] = 1
    return x

# Función que calcula la distancia al padre común mas cercano de dos elementos HTML
def get_parent_distance(s_cm, s_el):

   
    s_cm = s_cm[0].split(">")
    s_cm = [w.strip() for w in s_cm]
    s_cm_len = len(s_cm)
    
    s_el = s_el.split(">")
    s_el = [w.strip() for w in s_el]   
    s_el_len = len(s_el)
    
    count_parent = 0

    if s_cm_len > s_el_len:
        smaller = min(s_cm_len,s_el_len)
        s_cm = s_cm[:smaller]
        s_el = s_el[:smaller]
        count_parent = s_cm_len - s_el_len
    else:
        smaller = min(s_cm_len,s_el_len)
        s_cm = s_cm[:smaller]
        s_el = s_el[:smaller]
        
    count = smaller

    while s_cm != s_el:
        count_parent += 1
        smaller -= 1
        s_cm = s_cm[:smaller]
        s_el = s_el[:smaller]        

    return count_parent




#### Aplicamos las funciones

In [ ]:
df_f = pd.DataFrame()

unique_url = df['url'].unique()


for url in unique_url:
    df_u = df[df['url'] == url]
    if df_u.count()[0] == 0:
        continue
    if df_u[df_u['label'] == 'mensaje_cookies'].count()[0] == 0:
        continue
        
    df_u['text'] = df_u['text'].apply(lambda x: re.sub("[≡!@#$≡<>+%*()'-]|\n|\t", ' ', str(x)))
    df_u['text'] = df_u['text'].apply(lambda x: re.sub(' +', ' ',x))
    df_u['text_length'] = df_u['text'].apply(lambda x: len(x.split()))
    
    df_u['position_x_y'] = df_u['position_x_y'].apply(lambda x: x.replace("(",""))
    df_u['position_x_y'] = df_u['position_x_y'].apply(lambda x: x.replace(")",""))
    df_u['position_x'] = df_u['position_x_y'].apply(lambda x: x.split(",")[0])
    df_u['position_x'] = df_u['position_x'].astype(float)
    df_u['position_y'] = df_u['position_x_y'].apply(lambda x: x.split(",")[1])
    df_u['position_y'] = df_u['position_y'].astype(float)
    df_u['width_height'] = df_u['width_height'].apply(lambda x: x.replace("(",""))
    df_u['width_height'] = df_u['width_height'].apply(lambda x: x.replace(")",""))
    df_u['width'] = df_u['width_height'].apply(lambda x: x.split(",")[0])
    df_u['width'] = df_u['width'].astype(float)
    df_u['height'] = df_u['width_height'].apply(lambda x: x.split(",")[1])
    df_u['height'] = df_u['height'].astype(float)
    df_u['position_x'] = df_u.apply(lambda x: new_x(x) , axis=1)
    df_u['position_y'] = df_u.apply(lambda x: new_y(x) , axis=1)

    cm_y = df_u['position_y'][df_u['label'] == 'mensaje_cookies'].values[0]
    cm_x = df_u['position_x'][df_u['label'] == 'mensaje_cookies'].values[0]

    df_u['distance_cm'] = df_u.apply(lambda x: calculate_distance(x,cm_x,cm_y),axis=1)
    df_f = pd.concat([df_f,df_u])

# Los siguientes cálculos los hacemos con el dataframe entero, pero antes comprobamos que no está vacío.
if len(df_f.index) > 0:
    
    # Seleccionamos las columnas que queremos del dataframe principal (df_f) y las guardamos en df_d
    df_d = df_f[['css_selector','url','label','text_length','width','height','distance_cm','text','name']]
    
    # Aplicamos one hot enconding al texto
    df_d = df_d.apply(lambda x: is_in_one_hot(x),axis=1)
    
    # Calculamos la mínima distancia al elemento HTML padre común entre el elemento a probar y el elemento de mensaje de cookies.
    # Este cálculo lo volvemos a hacer por URl
    df_d['parent_distance'] = 0
    for url in df_d['url'].unique():
        df_url = df_d[df_d['url'] == url]
        cm = df_d[df_d['url'] == url][df_d['label'] == 'mensaje_cookies']
        cm_css_selector = cm['css_selector'].values
        df_url['parent_distance'] = df_url['css_selector'].apply(lambda x: get_parent_distance(cm_css_selector,x))
        df_d[df_d['url'] == url] = df_url
    
    df_d = df_d.fillna(0)

#### Normalización de datos

In [ ]:
# Normalize data
from sklearn import preprocessing

columns_to_normalize = ['text_length','width','height','distance_cm','parent_distance']

for col in columns_to_normalize:
    df_d[col]=(df_d[col]-df_d[col].min())/(df_d[col].max()-df_d[col].min())
    #df_d[col]=(df_d[col]-df_d[col].mean())/df_d[col].std()

In [ ]:
df_d.describe()

In [ ]:
df_d[df_d['label'].isin(['cerrar_cookies'])].describe()

In [ ]:
df_tt = df_d.copy()

df_tt[df_tt['name'] == 'a']['name'] = None

# Hacemos get dummies para crear las columnas one hot encoding de los atributos name de los elementos
df_tt = pd.get_dummies(df_tt, columns=['name'])

# Quitamos las columnas que no necesitamos
df_tt.drop('css_selector',axis=1, inplace=True)
df_tt.drop('url',axis=1, inplace=True)
df_tt.drop('text',axis=1, inplace=True)

# Seleccionamos un parte de los elementos que están categorizados como "cerrar_cookies"
df_tt_ok = df_tt[df_tt['label'].isin(['cerrar_cookies'])]
df_tt_ok_test = df_tt_ok.sample(frac=0.2)
df_tt_ok_train = pd.concat([df_tt_ok, df_tt_ok_test]).drop_duplicates(keep=False)

# Seleccionamos una parte de los elementos que no son "cerrar_cookies"
# Aquí dividimos en dos partes, para entrenar bien a nuestro modelo, le metemos elementos de cualquier sitio
# de la página y elemento que están cercanos al "mensaje_cookies" así le enseñamos a saber que 
# no por estar cerca es un elemento del tipo "cerrar_cookies"
df_tt_ko = df_tt[~df_tt['label'].isin(['cerrar_cookies'])]
df_tt_ko_near = df_tt_ko[df_tt_ko['parent_distance'] < 0.17].sample(frac=0.010)
df_tt_ko_far = df_tt_ko[df_tt_ko['parent_distance'] >= 0.17].sample(frac=0.003)



df_tt_ko = pd.concat([df_tt_ko_near, df_tt_ko_far]).sample(frac=0.6)



df_tt_ko_test = df_tt_ko.sample(frac=0.2)
df_tt_ko_train = pd.concat([df_tt_ko, df_tt_ko_test]).drop_duplicates(keep=False)


df_tt_test = pd.concat([df_tt_ok_test,df_tt_ko_test])
df_tt_test_label = df_tt_test.copy()['label']
df_tt_test.drop('label',axis=1, inplace=True)


df_tt_train = pd.concat([df_tt_ok_train,df_tt_ko_train])
df_tt_train_label = df_tt_train.copy()['label']
df_tt_train.drop('label',axis=1, inplace=True)

df_tt_total = pd.concat([df_tt_ok,df_tt_ko.sample(frac=0.001)])
df_tt_total_label = df_tt_total.copy()['label']
df_tt_total.drop('label',axis=1, inplace=True)


In [ ]:
# Guardamos en un pickle
import pickle

with open('df_cerrar_train.pkl', 'wb') as f:
    pickle.dump(df_tt_train, f)
    
with open('df_cerrar_test.pkl', 'wb') as f:
    pickle.dump(df_tt_test, f)
    
with open('df_cerrar_train_label.pkl', 'wb') as f:
    pickle.dump(df_tt_train_label, f)
    
with open('df_cerrar_test_label.pkl', 'wb') as f:
    pickle.dump(df_tt_test_label, f)